## Imports

In [1]:
import gc
import logging
import math
import random
import time
import warnings
from collections import defaultdict
from datetime import datetime
from importlib import reload
from itertools import product
from subprocess import Popen, PIPE
from threading import Thread
from warnings import catch_warnings, simplefilter

import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy.stats import qmc, norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
from torch.nn import *
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm

reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')

## Neural Network Classes

In [2]:
class SimpleCNN(Module):
    def __init__(self, config, verbose):
        super(SimpleCNN, self).__init__()
        img_dim = config['shape'][2]
        channels = config['shape'][1]
        conv_layers = np.array([
            [LazyConv2d(channels * 3 ** (depth + 1), kernel_size=3, stride=1, padding=1),
             LazyBatchNorm2d(),
             ReLU(inplace=True),
             MaxPool2d(kernel_size=2, stride=2)]
            for depth in range(config['factors'].get('nr_conv_layers', 2))
        ]).flatten().tolist()
        linear_layers = [
            LazyLinear(max(math.floor(channels * img_dim ** 2 / 2 ** (depth + 1)), channels)) for depth in
            range(config['factors'].get('nr_linear_layers', 3) - 1)
        ]
        self.layers = Sequential(
            *conv_layers,
            Flatten(start_dim=1),
            *linear_layers,
            LazyLinear(config['labels'])
        )
        if verbose:
            logging.info(self.layers)

    def forward(self, x):
        x = self.layers(x)
        return x


class DensePolyNN(Module):
    def __init__(self, config, verbose):
        super(DensePolyNN, self).__init__()

        img_dim = config['shape'][2]
        channels = config['shape'][1]
        linear_layers = [
            LazyLinear(max(math.floor(channels * img_dim ** 2 / 2 ** (depth + 1)), channels)) for depth in
            range(config['factors'].get('nr_linear_layers', 3) - 1)
        ]

        self.layers = Sequential(
            Flatten(start_dim=1),
            *linear_layers,
            LazyLinear(config['labels'])
        )
        if verbose:
            logging.info(self.layers)

    def forward(self, x):
        x = self.layers(x)
        return x


class DenseLinearNN(Module):
    def __init__(self, config, verbose):
        super(DenseLinearNN, self).__init__()

        img_dim = config['shape'][2]
        channels = config['shape'][1]
        linear_layers = [
            LazyLinear(max(math.floor(
                channels * img_dim ** 2 - (depth + 1) * (
                        channels * img_dim ** 2 / config['factors'].get('nr_linear_layers', 3))),
                channels)) for depth in range(config['factors'].get('nr_linear_layers', 3) - 1)]

        self.layers = Sequential(
            Flatten(start_dim=1),
            *linear_layers,
            LazyLinear(config['labels'])
        )
        if verbose:
            logging.info(self.layers)

    def forward(self, x):
        x = self.layers(x)
        return x

## Get GPU values

In [3]:
class GPUMonitor(Thread):
    def __init__(self, delay):
        super(GPUMonitor, self).__init__()
        self.delay = delay
        self.power_readings = []
        self.running = True
        self.start()

    def run(self):
        while self.running:
            try:
                p = Popen('nvidia-smi --query-gpu=power.draw --format=csv,noheader,nounits'.split(' '), stdout=PIPE)
                stdout, stderror = p.communicate()
                self.power_readings.append(float(stdout.strip()))
                p.terminate()
            except:
                logging.error('Something went wrong while retrieving GPU readings...')
            time.sleep(self.delay)

    def stop(self):
        self.running = False

    def reset_energy(self):
        self.power_readings = []

    def get_power_average(self):
        return np.mean(self.power_readings)


## Model operations

In [12]:
def create_model(net, config, verbose):
    model = net(config=config, verbose=verbose)
    optimizer = Adam(model.parameters(),
                     lr=config['factors'].get('learning_rate', 1e-3),
                     betas=(config['factors'].get('beta1', 0.9), config.get('beta2', 0.999)),
                     eps=config['factors'].get('epsilon', 1e-8),
                     weight_decay=config['factors'].get('weight_decay', 0)
                     )
    criterion = CrossEntropyLoss()
    if torch.cuda.is_available():
        if verbose:
            logging.info('Using GPU')
        model = model.cuda()
        criterion = criterion.cuda()
    return model, optimizer, criterion


def train(net, config, batches, tag, verbose=False):
    logging.info(f'Using {config["factors"]}')
    logging.info(f'Constructing {tag}')
    model, optimizer, criterion = create_model(net, config, verbose)
    train_losses = []
    logging.info('Training the model:')
    gpu_monitor = GPUMonitor(0.1)
    t_start = time.perf_counter()
    for _ in tqdm(range(config['epochs'])):
        for batch_id, batch in batches:
            train_x = batch[0]
            train_y = batch[1]
            model.train()
            train_x, train_y = Variable(train_x), Variable(train_y)
            if torch.cuda.is_available():
                train_x = train_x.cuda()
                train_y = train_y.cuda()

            # clearing the Gradients of the model parameters
            optimizer.zero_grad()

            # prediction for training set
            output_train = model(train_x)

            # computing the training loss
            loss_train = criterion(output_train, train_y)
            train_losses.append(loss_train.item())

            # computing the updated weights of all the model parameters
            loss_train.backward()
            optimizer.step()
    if verbose:
        plt.plot(train_losses, label='Training loss')
        plt.yscale('log')
        plt.legend()
        plt.show()
    avg_power = gpu_monitor.get_power_average()
    gpu_monitor.stop()
    return model, tag, config['factors'], avg_power, time.perf_counter() - t_start


def test(models, test_x, test_y):
    logging.info('Generating predictions and calculating accuracy')
    test_results = []
    for model, tag, factors, avg_power, duration in models:
        with torch.no_grad():
            output = model(test_x.cuda())

        softmax = torch.exp(output).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)
        accuracy = accuracy_score(test_y, predictions)
        test_results.append((tag, accuracy, factors, avg_power, duration))
        logging.info(f'{tag}: {accuracy=}')
    return test_results


def predict(model, test_x, predictions):
    logging.info('Generating predictions')
    with torch.no_grad():
        output = model(test_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions['label'] = np.argmax(prob, axis=1)
    return predictions

## Training strategies

In [5]:
def random_strategy(factors_bounds, random_function, tag):
    """
    Returns a configuration of (hyper)parameter values according to a random strategy
    :param factors_bounds: The lower and upper bound for all the values
    :param random_function: The method for generating random samples
    :param tag: NOT USED, PASS NONE VALUE
    :return: (hyper)parameter configuration
    """
    logging.info('Applying the random strategy...')

    factors = []
    lowers = []
    uppers = []
    is_ints = []
    for factor, (low, up, is_int) in factors_bounds.items():
        factors.append(factor)
        lowers.append(low)
        uppers.append(up)
        is_ints.append(is_int)

    return random_function(factors, 1, lowers, uppers, is_ints)[0]


def grid_search(factors_bounds, random_function, tag):
    """
    Returns a configuration of (hyper)parameter values according to a grid search strategy
    :param factors_bounds: The lower and upper bound for all the values
    :param random_function: NOT USED, PASS NONE VALUE
    :param tag: The tag of the neural network
    :return: (hyper)parameter configuration
    """
    logging.info('Applying a grid search strategy...')
    if tag not in grid_search_space:
        all_values = []
        size = 1
        real_value_split = 5
        for factor, (lower, upper, is_int) in factors_bounds.items():
            if is_int:
                all_values.append([lower + i for i in range(upper - lower)])
                size *= (upper - lower)
            else:
                all_values.append([lower + i * (upper - lower) / real_value_split for i in range(real_value_split)])
                size *= real_value_split
        search_space = list(product(*all_values))
        random_indices = random.sample(range(size), config['experiments'])
        grid_search_space[tag] = (random_indices, search_space)

    random_indices, search_space = grid_search_space[tag]
    factors = list(factors_bounds.keys())
    random_idx = random_indices.pop()
    return {factors[i]: search_space[random_idx][i] for i in range(len(factors))}


def bayesian_pi(factor_bounds, random_function, tag):
    """
    Returns a configuration of (hyper)parameter values according to a Bayesian strategy with probability of improvement
    :param factors_bounds: The lower and upper bound for all the values
    :param random_function: The method for generating random samples
    :param tag: The tag of the neural network
    :return: (hyper)parameter configuration
    """
    logging.info('Applying the bayesian strategy with probability of improvement...')

    factors = []
    lowers = []
    uppers = []
    is_ints = []
    for factor, (low, up, is_int) in factor_bounds.items():
        factors.append(factor)
        lowers.append(low)
        uppers.append(up)
        is_ints.append(is_int)

    # Return a random sample and initialize the regression model if it does not exist
    if tag not in regression_models:
        random_factors = random_function(factors, 1, lowers, uppers, is_ints)[0]
        regression_models[tag] = {
            "model": GaussianProcessRegressor(),
            "X": [],
            "y": []
        }
        return random_factors

    # Generate 10000 random samples and return the one with the highest probability of improvement, update the regression model
    regression_models[tag]['model'].fit(regression_models[tag]['X'], regression_models[tag]['y'])
    random_samples = random_function(factors, 10000, lowers, uppers, is_ints)
    candidate_sample = best_probability_of_improvement(factors, random_samples, regression_models[tag]['model'],
                                                       max(regression_models[tag]['y']))
    return candidate_sample

## Random functions

In [6]:
def get_quasi_random_samples(factors, length, lower, upper, is_int, sampler=qmc.Halton):
    sampler = sampler(len(factors))
    sample = sampler.random(length)
    return qmc.scale(sample, lower, upper)


def get_random_samples(factors, length, lower, upper, is_int):
    return [
        {factors[i]: round(random.uniform(lower[i], upper[i])) if is_int[i] else random.uniform(lower[i], upper[i]) for
         i in range(len(factors))} for _ in range(length)]

## Acquisition functions

In [7]:
def best_probability_of_improvement(factors, samples, model, y_best, maximize=True):
    best = -1
    x_next = None
    for x in samples:
        x = list(x.values())
        with catch_warnings():
            # ignore generated warnings
            simplefilter("ignore")
            mu_, std_ = model.predict([x], return_std=True)
            pi = norm.cdf((mu_ - y_best) / (std_ + 1e-9)) if maximize else norm.cdf((y_best - mu_) / (std_ + 1e-9))
            if pi > best:
                best = pi
                x_next = x
    return {factors[i]: x_next[i] for i in range(len(factors))}

## Configuration

In [8]:
# Configurable parameters:
# nr_linear_layers: default 3
# nr_conv_layers: default 2
# learning_rate: default 1e-3
# beta1: default 0.9
# beta2: default 0.999
# epsilon: default 1e-8
# weight_decay: default 0
config = {
    'experiments': 5,
    'repetitions': 1,
    'epochs': 10,
    'batch_size': 512,
    'dataset': datasets.FashionMNIST,
    'random_function': get_random_samples,
    'optimization_strategy': grid_search,
    'factor_bounds': {
        'nr_linear_layers': (1, 8, True),
        'learning_rate': (1e-5, 5e-3, False),
        'beta1': (0.4, 1, False),
        'beta2': (0.9, 1, False),
        'epsilon': (1e-9, 1e-7, False),
        'weight_decay': (0, 0.1, False)
    }
}

## Load data

In [9]:
logging.info('Loading data...')
training_data = config['dataset'](
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=config['batch_size'], shuffle=True)
batches = [(batch_id, batch) for batch_id, batch in enumerate(train_dataloader)]
logging.info('Training data loaded')

test_data = config['dataset'](
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)
test_x, test_y = next(iter(test_dataloader))
logging.info('Test data loaded')
config['shape'] = batches[0][1][0].shape
config['labels'] = len(training_data.classes)

02:09:42 INFO: Loading data...
02:09:46 INFO: Training data loaded
02:09:47 INFO: Test data loaded


## Train models

In [13]:
seed = 43
torch.manual_seed(seed)
random.seed(seed)
results = defaultdict(list)
regression_models = {}
grid_search_space = {}
####################
nets_to_train = [DensePolyNN, SimpleCNN]
####################
for experiment in range(config['experiments']):
    logging.info(f'\n\n#### RUNNING EXPERIMENT {experiment + 1}/{config["experiments"]} ####\n')
    models = []
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for net in nets_to_train:
            config['factors'] = config['optimization_strategy'](config['factor_bounds'], config['random_function'],
                                                                net.__name__)
            for _ in range(config['repetitions']):
                t_start = time.perf_counter()
                models.append(train(DensePolyNN, config, batches, net.__name__))

    test_results = test(models, test_x, test_y)
    for tag, acc, used_config, avg_power, duration in test_results:
        if tag in regression_models:
            regression_models[tag]['X'].append(list(used_config.values()))
            regression_models[tag]['y'].append(acc)
        results[tag].append((experiment, acc, used_config, avg_power, duration))
    gc.collect()
    torch.cuda.empty_cache()

02:11:23 INFO: 

#### RUNNING EXPERIMENT 1/5 ####

02:11:23 INFO: Applying a grid search strategy...
02:11:23 INFO: Using {'nr_linear_layers': 4, 'learning_rate': 0.004002, 'beta1': 0.52, 'beta2': 0.98, 'epsilon': 8.019999999999999e-08, 'weight_decay': 0.0}
02:11:23 INFO: Constructing DensePolyNN
02:11:23 INFO: Training the model:
100%|██████████| 10/10 [00:02<00:00,  4.15it/s]
02:11:25 INFO: Applying a grid search strategy...
02:11:25 INFO: Using {'nr_linear_layers': 7, 'learning_rate': 0.0010080000000000002, 'beta1': 0.88, 'beta2': 0.96, 'epsilon': 2.0799999999999998e-08, 'weight_decay': 0.0}
02:11:25 INFO: Constructing SimpleCNN
02:11:25 INFO: Training the model:


15


100%|██████████| 10/10 [00:03<00:00,  2.67it/s]
02:11:29 INFO: Generating predictions and calculating accuracy
02:11:29 INFO: DensePolyNN: accuracy=0.7924
02:11:29 INFO: SimpleCNN: accuracy=0.8339
02:11:29 INFO: 

#### RUNNING EXPERIMENT 2/5 ####

02:11:29 INFO: Applying a grid search strategy...
02:11:29 INFO: Using {'nr_linear_layers': 5, 'learning_rate': 0.004002, 'beta1': 0.52, 'beta2': 0.92, 'epsilon': 2.0799999999999998e-08, 'weight_decay': 0.02}
02:11:29 INFO: Constructing DensePolyNN
02:11:29 INFO: Training the model:


22


100%|██████████| 10/10 [00:03<00:00,  3.22it/s]
02:11:32 INFO: Applying a grid search strategy...
02:11:32 INFO: Using {'nr_linear_layers': 6, 'learning_rate': 0.0010080000000000002, 'beta1': 0.4, 'beta2': 0.96, 'epsilon': 1e-09, 'weight_decay': 0.04}
02:11:32 INFO: Constructing SimpleCNN
02:11:32 INFO: Training the model:


19


 10%|█         | 1/10 [00:00<00:02,  3.28it/s]
02:11:33 ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.

02:11:33 INFO: 
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_19004/3660253868.py", line 20, in <module>
    models.append(train(DensePolyNN, config, batches, net.__name__))
  File "/tmp/ipykernel_19004/2711197810.py", line 48, in train
    optimizer.step()
  File "/home/yarally1/.local/lib/python3.8/site-packages/torch/optim/optimizer.py", line 88, in wrapper
    return func(*args, **kwargs)
  File "/home/yarally1/.local/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    with self.__class__():
  File "/home/yarally1/.local/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 120, in __init__
    if not torch._jit_internal.is_scripting():
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/py

TypeError: object of type 'NoneType' has no len()

## Write Results

In [11]:
file_name = f'exp_results_{datetime.today().strftime("%d%m%Y_%H%M%S")}.csv'
file = open(f'results/{file_name}', 'a')
for tag in results:
    print(f'{tag}:')
    file.write(f'{tag}:\n')
    print(f'experiment,accuracy,avg_power,duration,{"".join(key + "," for key in results[tag][0][2])[:-1]}')
    file.write(f'experiment,accuracy,avg_power,duration,{"".join(key + "," for key in results[tag][0][2])[:-1]}\n')
    [print(f'{experiment},{acc},{avg_power},{duration},{"".join(str(val) + "," for val in used_config.values())[:-1]}') for
     experiment, acc, used_config, avg_power, duration in results[tag]]
    file.writelines([f'{experiment},{acc},{avg_power},{duration},{"".join(str(val) + "," for val in used_config.values())[:-1]}\n' for
                     experiment, acc, used_config, avg_power, duration in results[tag]])
file.close()
logging.info(f'Results logged to: results/{file_name}')

02:10:15 INFO: Results logged to: results/exp_results_27122021_141015.csv


DensePolyNN:
experiment,accuracy,avg_power,duration,nr_linear_layers,learning_rate,beta1,beta2,epsilon,weight_decay
0,0.7924,66.79586206896552,4.466207781999401,4,0.004002,0.52,0.98,8.019999999999999e-08,0.0
1,0.7415,82.975,2.821976294999331,5,0.004002,0.52,0.92,2.0799999999999998e-08,0.02
2,0.7498,82.57833333333333,1.9026276290005626,2,0.0020060000000000004,0.64,0.96,6.04e-08,0.02
3,0.7068,85.83857142857143,2.2008476829996653,3,0.004002,0.88,0.98,8.019999999999999e-08,0.08
4,0.7354,65.91363636363637,1.8324149449999823,1,0.0020060000000000004,0.4,0.9,4.06e-08,0.06000000000000001
SimpleCNN:
experiment,accuracy,avg_power,duration,nr_linear_layers,learning_rate,beta1,beta2,epsilon,weight_decay
0,0.8339,81.48263157894738,3.099681106999924,7,0.0010080000000000002,0.88,0.96,2.0799999999999998e-08,0.0
1,0.652,81.73315789473682,3.0113047750000987,6,0.0010080000000000002,0.4,0.96,1e-09,0.04
2,0.7258,79.48523809523809,3.300003373000436,7,0.0010080000000000002,0.52,0.98,8.019999999999999e-08,0.02